## KDVEM k-anonymity
- a k-degree anonymity with with vertex and edge modification algorithm

I am choosing to implement this despite it beign a k-anon method. Intuitively to me it shoudl produce the same results as the ANO-NET k-anon methods, but there might be fairness differences in tthe implementation, even if the target statistic is the same.

- k-degree anonymous when each vertex in the graph has the same degree as at least k-1 other vertices
- done by adding noise edges and vertices, try to respect community structure and path length
    - might be the same as k-deg sequence anon, check paper if I have time
- social distance btwn all pairs in measured by Average Path Length = sum_over(u,v) ( SPL(u,v)) / |RP| where |RP| is reachable pair vertices
- two step process
    - compute target degree for each node to make graph k-anon
    - change vertex deg to target by addign edges / vertices

- deg sequence has not only deg info but also ID info -> they yse d[i] to represent the tuple (Vj, deg_Vj) where j is the node with the ith highest degree
- k-anon deg sequence has n subsequences, where each has at least k els, and all have the same deg

In [1]:
import numpy as np
import networkx as nx
import random

## we use the DD199 graph for testing
import pandas as pd
DD199 = nx.read_edgelist("Data/DD199/DD199.edges", nodetype=int)

BAG = nx.barabasi_albert_graph(20,2)

In [ ]:
def greedy_partition(degDict, k):
    '''original deg seq d, anon level k, 
    Rerturns targetDegDict = node: targetDeg, and dprime = targetDeg : [nodes]'''

    d = sorted(degDict.items(), key = lambda item : item[1], reverse=True)  ## list of (node:degree)

    dprime = {d[0][1]:[pair[0] for pair in d[0:k]]}    ## {target degree: [nodes]}, initialized with maxDeg : [k largest deg nodes]

    target_d = d[0][1]
    i = k
    while i < len(d)-k:  ## iterate through the degree pairs, except the last k (they are necc. added to a group)
        (vi, dvi) = d[i]
        ## compute costs to merge and to make new group
        ## since we do addition to make series same, cost_merge= maxDeg_i-deg v and cost_new = k*deg_v - sum(deg_v+1,...,deg_v+k)
        C_merge = target_d - dvi
        C_new = k*dvi - sum([pair[1] for pair in d[i:i+k]]) ## sum of next k degrees
        if C_merge <= C_new:        ## default has to be merge otherwise therea re issues
            dprime[target_d].append(vi)
            i+= 1
        else:
            target_d = dvi
            dprime[target_d] = [pair[0] for pair in d[i:i+k]]    ## get next k nodes in deg sequence and assign to dprime with key= current deg
            i += k 
    
    dprime[target_d] = dprime[target_d] + [pair[0] for pair in d[i:]]
    
    targetDegDict = {node: targetDeg for targetDeg in dprime for node in dprime[targetDeg]}  ## 

    return targetDegDict, dprime

In [ ]:
def find_candidates(G:nx.Graph, degDef):
    '''     Creates a list of candidates for each vertex
    INPUT: G graph, degDef = {node: degree deficiency}
    Returns dictionary node: [(candidate, dist cand to node)] NOTE: contains only nodes with deg deficiency > 0

    iterate through a vertexes comm by increasing level (full graph at top), 
        add u in comm to temp, if the (u,v) does not exist; sort temp by increasing social distance between u,v; add temp to candidates
    candidates is sorted first by lowest community level, then by lowest social distance
    NOTE: implemented with two levels of community, and with different comm detection than in paper
    '''
    commSets = nx.community.greedy_modularity_communities(G)
    ## lvl0Comm is full graph, so all nodes in same community i.e. node:0 for all nodes
    lvl1Comm = {i:j for j in range(len(commSets)) for i in commSets[j]}

    candidates = {}     ## node: [(Cand Node,dist)]
    for v in G.nodes():
        if degDef[v]<= 0:
            ## candidates[v] = []
            continue
        temp_v = []     ## (node, distance)
        for u in commSets[lvl1Comm[v]]:     ## 6
            if u!=v and (u,v) not in G.edges() and degDef[u] >0:     ## 5 /6
                temp_v.append((u,nx.shortest_path_length(G,u,v)))
            ## end if
        ## end for
        temp_v = sorted(temp_v, key = lambda item:item[1])  ## 12
        candidates[v] = temp_v
        temp_v = []
        for u in G.nodes():     ## 6
            if u!=v and (u,v) not in G.edges() and degDef[u] >0:     ## 5 /6
                temp_v.append((u,nx.shortest_path_length(G,u,v)))
            ## end if
        ## end for
        temp_v = sorted(temp_v, key = lambda item:item[1])  ## 12
        candidates[v] = candidates[v] + temp_v

    return candidates

In [104]:
def graph_modification(G:nx.Graph, k, d, d_target, dprime):
    '''Transform into k-anon graph
    use community_multilevel method in igraph, whole graph as highest level community
    for each vertex that needs deg increase, try to connect to candidates within the original graph
    candidate needs deg increase, and (v, u) doesnt exist yet, candidate vertex will be shrunk to the same community as low level as '''

    Gprime = G.copy()
    degSeq = sorted(d.items(), key = lambda item : item[1], reverse=True)  ## list of (node:degree)
    degDeficiency = {node: d_target[node]-d[node] for node in G.nodes()}

    candidates = find_candidates(G, degDeficiency)                  ## 11
    ## in pseud.Code candidates = [(node,distance)] & are found indiv.      Vplus= [(node, deficit)]

    for v in G.nodes():                                             ## 9
        ## work directly with degDeficiency not in temp
        while degDeficiency[v] > 0:                                 ## 13
            if len(candidates[v]) <=0:                              ## 18
                break   ## exit while loop
            candidate = candidates[v].pop(0)    ## (node, distance) ## 14
            Gprime.add_edge(v, candidate[0])
            degDeficiency[v] -= 1
        ## end while                                                ## 21
        ## update Vplus                                             ## 22-26
    ## end for
    
    ## addVertex following Chester [22]
    ## add m = (1+max(md,k))(mod2) + max(md,k)     md = di - dj where di largest deg, dj smallest deg, within any partition -> so largest val in deg Def
    md = max(degDeficiency.values())
    m = (1+max(md,k))%2 + max(md,k)
    newNodes = list(range(Gprime.number_of_nodes(),Gprime.number_of_nodes()+m))
    Gprime.add_nodes_from(newNodes)

    ## iterate through new nodes, and add edge to old nodes -> cycle through these slower
    newNodeID = 0
    for (node, deg) in degSeq:
        for i in range(degDeficiency[node]):
            Gprime.add_edge(node, newNodes[newNodeID%len(newNodes)])
            newNodeID +=1
    d = int(newNodeID/len(newNodes)) + 1

    if (d in d_target.values() and d-1 in d_target.values()) or newNodeID == 0: ## also k-anon if all new nodes have same deg
        return Gprime
    ## vertices with deg d-1 pair them and add an edge between them
    leftoverNodes = newNodes[newNodeID%len(newNodes):]      ## list of length m-td mod m with nodes of degree d-1
    random.shuffle(leftoverNodes)
    for i in range(0,len(leftoverNodes)-1,2):
        Gprime.add_edge(leftoverNodes[i],leftoverNodes[i+1])
    ## that is [newNodeID to end of list] of new nodes -> if m-td mod m is even and m >= k then we are done
    if len(leftoverNodes)%2 ==0:
        return Gprime
    ## if m-td mod d is odd, then we leave out 1 vertex -> m-1 even and >=k 
    ## add an edge from r to two other new vertices (from full list) -> have degree
    edgesDeg_d = newNodes[0:newNodeID%len(newNodes)] + leftoverNodes[:-1]   ## all but the last element
    random.shuffle(edgesDeg_d)
    Gprime.add_edges_from([(leftoverNodes[-1],edgesDeg_d[0]),(leftoverNodes[-1],edgesDeg_d[1])])
    for i in range(2,len(edgesDeg_d)-1,2):  ## pair off, but els 0 and 1 are already taken
        Gprime.add_edge(edgesDeg_d[i],edgesDeg_d[i+1])
    
    return Gprime

In [102]:
def KDVEMapproach(G:nx.Graph, k = 10):
    '''
    Performs a modification to G(V,E) to make it anonymois. Node mod is done only when edge mod cannot succeed
    '''
    ## default k is 10, bc that is a value they tested in the paper
    d = dict(G.degree())
    target_deg_dict, dprime = greedy_partition(d, k)
    Ganon = graph_modification(G, k,d, target_deg_dict, dprime)
    return Ganon